![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [139]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [140]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

data types and contents of a column

In [141]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [142]:
columns = ds_jobs_transformed.columns
for c in columns:
    print(ds_jobs_transformed.value_counts(c, dropna = False))
    print("\n")

student_id
1        1
22486    1
22484    1
22483    1
22482    1
        ..
11366    1
11364    1
11363    1
11362    1
33380    1
Length: 19158, dtype: int64


city
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_111       3
city_129       3
city_121       3
city_171       1
city_140       1
Length: 123, dtype: int64


city_development_index
0.920    5200
0.624    2702
0.910    1533
0.926    1336
0.698     683
         ... 
0.649       4
0.824       4
0.625       3
0.781       3
0.664       1
Length: 93, dtype: int64


gender
Male      13221
NaN        4508
Female     1238
Other       191
dtype: int64


relevant_experience
Has relevant experience    13792
No relevant experience      5366
dtype: int64


enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
NaN                   386
dtype: int64


education_level
Graduate          11598
Masters            4361
High School        2017


Converting integers, floats, and unordered categories

Converting integers, floats

In [143]:
for c in columns:
    if ds_jobs_transformed[c].dtype == "int64":
        ds_jobs_transformed[c] = ds_jobs_transformed[c].astype("int32")
    elif ds_jobs_transformed[c].dtype == "float64":
        ds_jobs_transformed[c] = ds_jobs_transformed[c].astype("float16")
        
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int32  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float16
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int32  
 13  job_change              19158 non-null  float16
dtypes: float16(2), int32(2), object(10)
me

Converting unordered categories : nominal categories to the category type, and two-factor categories to the bool type.

In [144]:
ds_jobs_transformed['city'] = ds_jobs_transformed['city'].astype("category")
ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].astype("category")
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].astype("bool")
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype("bool")
ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].astype("category")
ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].astype("category")

ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  object  
 6   education_level         18698 non-null  object  
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  object  
 9   company_size            13220 non-null  object  
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  object  
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(4

**Converting ordered categories**

Convert columns containing ordinal categorical data into ordered categories.

In [145]:
# Colums containing ordinal cateogrical data : enrolled_university, education_level, experience, company_size, last_new_job
print(ds_jobs_transformed["enrolled_university"].value_counts(dropna = False))
enrolled_university_order = ["no_enrollment", "Part time course", "Full time course"]
print("\n")

print(ds_jobs_transformed["education_level"].value_counts(dropna = False))
education_level_order = ["Primary School", "High School", "Graduate", "Masters", "Phd"]
print("\n")

print(ds_jobs_transformed["experience"].value_counts(dropna = False))
oneToTwentyExp = [str(exp) for exp in range(1, 21)]
experience_order = ["<1"] + oneToTwentyExp + [">20"]
print("\n")

print(ds_jobs_transformed["company_size"].value_counts(dropna = False))
company_size_order = ["<10", "10-49", "50-99", "100-499", "500-999", "1000-4999", "5000-9999", "10000+"]
print("\n")

print(ds_jobs_transformed["last_new_job"].value_counts(dropna = False))
last_new_job_order = ["never", "1", "2", "3", "4", ">4"]

no_enrollment       13817
Full time course     3757
Part time course     1198
NaN                   386
Name: enrolled_university, dtype: int64


Graduate          11598
Masters            4361
High School        2017
NaN                 460
Phd                 414
Primary School      308
Name: education_level, dtype: int64


>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
NaN      65
Name: experience, dtype: int64


NaN          5938
50-99        3083
100-499      2571
10000+       2019
10-49        1471
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
Name: company_size, dtype: int64


1        8040
>4       3290
2        2900
never    2452
4        1029
3        1024
NaN       423
Name: last_new_job, dtype: int64


In [146]:
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype("category")
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].astype("category")
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].astype("category")
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].astype("category")
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].astype("category")

In [147]:
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].cat.reorder_categories(enrolled_university_order, ordered=True)
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].cat.reorder_categories(education_level_order, ordered=True)
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].cat.reorder_categories(experience_order, ordered=True)
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].cat.reorder_categories(company_size_order, ordered=True)
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].cat.reorder_categories(last_new_job_order, ordered=True)

ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(9

**Filtering on ordered categorical columns**

Filter the DataFrame to only contain students with 10 or more years of experience at companies with at least 1000 employees.

In [148]:
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed["experience"] >= "10") & (ds_jobs_transformed["company_size"] >= "1000-4999")]
print(ds_jobs_transformed.head())
ds_jobs_transformed.shape

    student_id      city  ...  training_hours job_change
9          699  city_103  ...             123      False
12       25619   city_61  ...              23      False
31       22293  city_103  ...             141      False
34       26494   city_16  ...             145      False
40        2547  city_114  ...              14      False

[5 rows x 14 columns]


(2201, 14)